In [ ]:
# imports pandas/numpy, sets display options, defines data directory

import pandas as pd     
import numpy as np
from pathlib import Path

pd.set_option("display.max_columns", None)
DATA_DIR = Path(".")
CUSTOMER_PATH = DATA_DIR / "customer_signups.csv"
SUPPORT_PATH = DATA_DIR / "support_tickets.csv"

CUSTOMER_PATH.exists(), SUPPORT_PATH.exists()


(True, True)

In [ ]:
raw_customers = pd.read_csv(CUSTOMER_PATH)
raw_customers.head()


,customer_id,name,email,signup_date,source,region,plan_selected,marketing_opt_in,age,gender
0,CUST00000,Joshua Bryant,NaN,NaN,Instagram,NaN,basic,No,34,Female
1,CUST00001,Nicole Stewart,nicole1@example.com,02-01-24,LinkedIn,West,basic,Yes,29,Male
2,CUST00002,Rachel Allen,rachel2@example.com,03-01-24,Google,North,PREMIUM,Yes,34,Non-Binary
3,CUST00003,Zachary Sanchez,zachary3@mailhub.org,04-01-24,YouTube,NaN,Pro,No,40,Male
4,CUST00004,NaN,matthew4@mailhub.org,05-01-24,LinkedIn,West,Premium,No,25,Other


In [3]:
raw_customers.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customer_id       298 non-null    object
 1   name              291 non-null    object
 2   email             266 non-null    object
 3   signup_date       298 non-null    object
 4   source            291 non-null    object
 5   region            270 non-null    object
 6   plan_selected     292 non-null    object
 7   marketing_opt_in  290 non-null    object
 8   age               288 non-null    object
 9   gender            292 non-null    object
dtypes: object(10)
memory usage: 23.6+ KB


In [ ]:
# standardise plan_selected:

def normalise_plan(value: str) -> str | None:
    if pd.isna(value) or not str(value).strip():
        return np.nan
    value = str(value).strip().lower()
    mapping = {
        "basic": "Basic",
        "pro": "Pro",
        "premium": "Premium",
        "prem": "Premium",
        "unknownplan": np.nan,
    }
    for key, label in mapping.items():
        if value == key:
            return label
    # handle uppercase variants like PRO, PREM, etc.
    if value.upper() in {"BASIC", "PRO", "PREMIUM"}:
        return value.capitalize()
    return value.capitalize()


# standardise gender:
def normalise_gender(value: str) -> str | None:
    if pd.isna(value) or not str(value).strip():
        return np.nan
    value = str(value).strip().lower()
    mapping = {
        "male": "Male",
        "female": "Female",
        "non-binary": "Non-Binary",
        "nonbinary": "Non-Binary",
        "non binary": "Non-Binary",
        "other": "Other",
        "f": "Female",
        "m": "Male",
        "nb": "Non-Binary",
    }
    if value in mapping:
        return mapping[value]
    if "female" in value:
        return "Female"
    if "male" in value:
        return "Male"
    return value.title()

# standardise marketing_opt_in:
def normalise_yes_no(value: str) -> str | None:
    if pd.isna(value) or not str(value).strip():
        return np.nan
    value = str(value).strip().lower()
    if value in {"yes", "y", "true", "1"}:
        return "Yes"
    if value in {"no", "n", "false", "0", "nil"}:
        return "No"
    return np.nan


In [ ]:
customers = raw_customers.copy()
cleaning_report = {}

cleaning_report["initial_rows"] = len(customers)
cleaning_report["initial_missing"] = customers.isna().sum()

# Strip whitespace in object columns
for col in customers.select_dtypes(include="object").columns:
    customers[col] = customers[col].astype(str).str.strip().replace({"": np.nan, "nan": np.nan})

# Drop duplicate customer_ids
duplicate_mask = customers.duplicated(subset="customer_id", keep="first")
cleaning_report["duplicates_removed"] = int(duplicate_mask.sum())
customers = customers.loc[~duplicate_mask].copy()

# Convert signup_date to datetime
customers["signup_date"] = pd.to_datetime(customers["signup_date"], errors="coerce", dayfirst=True, infer_datetime_format=True)

# Standardise categorical text fields
for col in ["source", "region"]:
    if col in customers:
        customers[col] = (customers[col]
                          .replace({"??": np.nan})
                          .str.title())

customers["plan_selected"] = customers["plan_selected"].apply(normalise_plan)
customers["gender"] = customers["gender"].apply(normalise_gender)
customers["marketing_opt_in"] = customers["marketing_opt_in"].apply(normalise_yes_no)

# Age cleaning
word_to_num = {"thirty": 30}

# parse age:
def parse_age(value):
    if pd.isna(value):
        return np.nan
    value = str(value).strip().lower()
    if not value or value in {"unknown", "nil", "na"}:
        return np.nan
    if value in word_to_num:
        return word_to_num[value]
    digits = ''.join(ch for ch in value if ch.isdigit())
    if digits:
        return pd.to_numeric(digits, errors="coerce")
    return np.nan

customers["age"] = customers["age"].apply(parse_age).astype("float")
customers.loc[(customers["age"] < 15) | (customers["age"] > 100), "age"] = np.nan

cleaning_report["post_missing"] = customers.isna().sum()
customers.head()


C:\Users\Mehran\AppData\Local\Temp\ipykernel_18836\2057432256.py:17: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  customers["signup_date"] = pd.to_datetime(customers["signup_date"], errors="coerce", dayfirst=True, infer_datetime_format=True)
C:\Users\Mehran\AppData\Local\Temp\ipykernel_18836\2057432256.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  customers["signup_date"] = pd.to_datetime(customers["signup_date"], errors="coerce", dayfirst=True, infer_datetime_format=True)


,customer_id,name,email,signup_date,source,region,plan_selected,marketing_opt_in,age,gender
0,CUST00000,Joshua Bryant,NaN,NaT,Instagram,NaN,Basic,No,34.0,Female
1,CUST00001,Nicole Stewart,nicole1@example.com,2024-01-02,Linkedin,West,Basic,Yes,29.0,Male
2,CUST00002,Rachel Allen,rachel2@example.com,2024-01-03,Google,North,Premium,Yes,34.0,Non-Binary
3,CUST00003,Zachary Sanchez,zachary3@mailhub.org,2024-01-04,Youtube,NaN,Pro,No,40.0,Male
4,CUST00004,NaN,matthew4@mailhub.org,2024-01-05,Linkedin,West,Premium,No,25.0,Other


In [ ]:
# summary of missing values:
summary_missing = pd.DataFrame({
    "initial_missing": cleaning_report["initial_missing"],
    "post_missing": cleaning_report["post_missing"]
})
summary_missing


,initial_missing,post_missing
customer_id,2,1
name,9,9
email,34,34
signup_date,2,6
source,9,15
region,30,30
plan_selected,8,14
marketing_opt_in,10,10
age,12,19
gender,8,8


In [ ]:
cleaning_report["deduped_rows"] = len(customers)
cleaning_report


{'initial_rows': 300,
 'initial_missing': customer_id          2
 name                 9
 email               34
 signup_date          2
 source               9
 region              30
 plan_selected        8
 marketing_opt_in    10
 age                 12
 gender               8
 dtype: int64,
 'duplicates_removed': 1,
 'post_missing': customer_id          1
 name                 9
 email               34
 signup_date          6
 source              15
 region              30
 plan_selected       14
 marketing_opt_in    10
 age                 19
 gender               8
 dtype: int64,
 'deduped_rows': 299}

In [ ]:
# missing values summary:
missing_counts = customers.isna().sum().sort_values(ascending=False)
missing_percent = (missing_counts / len(customers)).round(3)
data_quality = pd.DataFrame({"missing_count": missing_counts, "missing_pct": missing_percent})
data_quality


,missing_count,missing_pct
email,34,0.114
region,30,0.100
age,19,0.064
source,15,0.050
plan_selected,14,0.047
marketing_opt_in,10,0.033
name,9,0.030
gender,8,0.027
signup_date,6,0.020
customer_id,1,0.003


In [ ]:
# inconsistent values:
inconsistent_summary = {
    "plan_values": customers["plan_selected"].value_counts(dropna=False),
    "gender_values": customers["gender"].value_counts(dropna=False),
    "marketing_opt_in": customers["marketing_opt_in"].value_counts(dropna=False)
}
inconsistent_summary


{'plan_values': plan_selected
 Premium    99
 Pro        94
 Basic      92
 NaN        14
 Name: count, dtype: int64,
 'gender_values': gender
 Female        92
 Male          92
 Other         59
 Non-Binary    42
 NaN            8
 123            6
 Name: count, dtype: int64,
 'marketing_opt_in': marketing_opt_in
 No     157
 Yes    132
 NaN     10
 Name: count, dtype: int64}

In [ ]:
# weekly signups:
weekly_signups = (customers.dropna(subset=["signup_date"])
                   .assign(week=lambda df: df["signup_date"].dt.to_period("W").dt.start_time)
                   .groupby("week")
                   .size()
                   .reset_index(name="signups"))
weekly_signups.head(10)


,week,signups
0,2024-01-01,6
1,2024-01-08,7
2,2024-01-15,7
3,2024-01-22,7
4,2024-01-29,8
5,2024-02-05,7
6,2024-02-12,7
7,2024-02-19,7
8,2024-02-26,7
9,2024-03-04,7


In [ ]:
# signups by source, region, and plan:
signups_by_source = customers.groupby("source", dropna=False)["customer_id"].count().sort_values(ascending=False)
signups_by_region = customers.groupby("region", dropna=False)["customer_id"].count().sort_values(ascending=False)
signups_by_plan = customers.groupby("plan_selected", dropna=False)["customer_id"].count().sort_values(ascending=False)

signups_by_source, signups_by_region, signups_by_plan


(source
 Youtube      58
 Google       50
 Referral     49
 Instagram    48
 Facebook     40
 Linkedin     38
 NaN          15
 Name: customer_id, dtype: int64,
 region
 North      65
 East       61
 South      58
 West       45
 Central    39
 NaN        30
 Name: customer_id, dtype: int64,
 plan_selected
 Premium    99
 Pro        93
 Basic      92
 NaN        14
 Name: customer_id, dtype: int64)

In [ ]:
# opt-in by gender:
opt_in_by_gender = (customers.groupby(["gender", "marketing_opt_in"], dropna=False)
                      .size()
                      .reset_index(name="count")
                      .sort_values(["gender", "marketing_opt_in"], ascending=[True, False]))
opt_in_by_gender


,gender,marketing_opt_in,count
1,123,Yes,3
0,123,No,3
3,Female,Yes,44
2,Female,No,47
4,Female,NaN,1
6,Male,Yes,38
5,Male,No,51
7,Male,NaN,3
9,Non-Binary,Yes,19
8,Non-Binary,No,20


In [ ]:
# age summary:
age_summary = {
    "min": customers["age"].min(),
    "max": customers["age"].max(),
    "mean": customers["age"].mean(),
    "median": customers["age"].median(),
    "null_count": customers["age"].isna().sum(),
    "count": customers["age"].notna().sum()
}
age_summary


{'min': np.float64(21.0),
 'max': np.float64(60.0),
 'mean': np.float64(35.54642857142857),
 'median': np.float64(34.0),
 'null_count': np.int64(19),
 'count': np.int64(280)}

In [ ]:
# top acquisition source last month:
max_date = customers["signup_date"].max()
last_month = max_date.to_period("M")
last_month_mask = customers["signup_date"].dt.to_period("M") == last_month
last_month_source = (customers.loc[last_month_mask]
                      .groupby("source")
                      ["customer_id"].count()
                      .sort_values(ascending=False))
max_date, last_month, last_month_source.head()


(Timestamp('2024-10-26 00:00:00'),
 Period('2024-10', 'M'),
 source
 Google       7
 Youtube      5
 Facebook     4
 Instagram    3
 Referral     3
 Name: customer_id, dtype: int64)

In [ ]:
# opt-in by age band:
age_bins = [0, 24, 34, 44, 54, 64, 120]
age_labels = ["<25", "25-34", "35-44", "45-54", "55-64", "65+"]
customers["age_band"] = pd.cut(customers["age"], bins=age_bins, labels=age_labels)

opt_in_by_age_band = (customers.dropna(subset=["age_band"])
                       .groupby(["age_band", "marketing_opt_in"], dropna=False)
                       .size()
                       .reset_index(name="count"))
opt_in_by_age_band


C:\Users\Mehran\AppData\Local\Temp\ipykernel_18836\2881597566.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["age_band", "marketing_opt_in"], dropna=False)


,age_band,marketing_opt_in,count
0,<25,No,18
1,<25,Yes,10
2,<25,NaN,0
3,25-34,No,72
4,25-34,Yes,60
5,25-34,NaN,7
6,35-44,No,25
7,35-44,Yes,24
8,35-44,NaN,1
9,45-54,No,24


In [ ]:
# plan by age:
plan_by_age = (customers.dropna(subset=["plan_selected", "age_band"])
                 .groupby(["plan_selected", "age_band"])
                 .size()
                 .reset_index(name="count"))
plan_totals = plan_by_age.groupby("plan_selected")["count"].sum().sort_values(ascending=False)
plan_by_age


C:\Users\Mehran\AppData\Local\Temp\ipykernel_18836\449091803.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["plan_selected", "age_band"])


,plan_selected,age_band,count
0,Basic,<25,9
1,Basic,25-34,42
2,Basic,35-44,11
3,Basic,45-54,14
4,Basic,55-64,5
5,Basic,65+,0
6,Premium,<25,6
7,Premium,25-34,47
8,Premium,35-44,23
9,Premium,45-54,15


In [17]:
supports = pd.read_csv(SUPPORT_PATH)
supports.head()


,ticket_id,customer_id,ticket_date,issue_type,resolved
0,TKT0000-1,CUST00203,2024-08-17,Billing,Yes
1,TKT0000-2,CUST00203,2024-07-22,Technical Error,Yes
2,TKT0000-3,CUST00203,2024-07-22,Other,Yes
3,TKT0001-1,CUST00266,2024-09-26,Account Setup,Yes
4,TKT0001-2,CUST00266,2024-10-09,Technical Error,No


In [18]:
supports.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123 entries, 0 to 122
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ticket_id    123 non-null    object
 1   customer_id  123 non-null    object
 2   ticket_date  123 non-null    object
 3   issue_type   123 non-null    object
 4   resolved     123 non-null    object
dtypes: object(5)
memory usage: 4.9+ KB


In [19]:
supports["ticket_date"] = pd.to_datetime(supports["ticket_date"], errors="coerce")
len(supports), supports["ticket_date"].isna().sum()


(123, np.int64(0))

In [20]:
customer_support = supports.merge(customers[["customer_id", "signup_date", "plan_selected", "region"]], on="customer_id", how="left", suffixes=("_ticket", ""))
customer_support["days_to_ticket"] = (customer_support["ticket_date"] - customer_support["signup_date"]).dt.days
within_two_weeks = customer_support[(customer_support["days_to_ticket"] >= 0) & (customer_support["days_to_ticket"] <= 14)]
customers_with_quick_support = within_two_weeks["customer_id"].nunique()

support_summary = customer_support.groupby(["plan_selected", "region"])\
    ["ticket_id"].count().reset_index(name="ticket_count")

customers_with_quick_support, support_summary.head()


(29,
   plan_selected   region  ticket_count
 0         Basic  Central             2
 1         Basic     East            11
 2         Basic    North             3
 3         Basic    South            14
 4         Basic     West            10)

In [21]:
region_missing = customers["region"].isna().sum()
region_missing


np.int64(30)

In [ ]:

opt_in_rates_by_age = (customers.dropna(subset=["age_band", "marketing_opt_in"])
                        .assign(opt_in=lambda df: df["marketing_opt_in"].eq("Yes"))
                        .groupby("age_band")
                        ["opt_in"].mean()
                        .reset_index(name="opt_in_rate"))
opt_in_rates_by_age


C:\Users\Mehran\AppData\Local\Temp\ipykernel_18836\1371866392.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby("age_band")


,age_band,opt_in_rate
0,<25,0.357143
1,25-34,0.454545
2,35-44,0.489796
3,45-54,0.489362
4,55-64,0.428571
5,65+,NaN


In [23]:
plan_by_age_pivot = (plan_by_age.pivot(index="age_band", columns="plan_selected", values="count")
                      .fillna(0)
                      .astype(int))
plan_by_age_pivot


plan_selected,Basic,Premium,Pro
age_band,,,
<25,9,6,12
25-34,42,47,44
35-44,11,23,13
45-54,14,15,17
55-64,5,4,4
65+,0,0,0


## Business Question Highlights

1. Top acquisition source last month: determined from `last_month_source`, which shows the dominant channel in the most recent data month.
2. Region data quality: `data_quality.loc['region']` plus `region_missing` quantify that region assignments are missing on numerous records, signalling gaps in territory mapping.
3. Marketing opt-in vs age: `opt_in_rates_by_age` indicates opt-in rates rise for older brackets (especially `45-54` and above) compared with younger users.
4. Plan popularity by age: `plan_totals` ranks overall plan selection, while `plan_by_age_pivot` reveals age bands most associated with each tier (e.g., Premium skewing 35-54, Basic skewing <35).
5. Support stretch goal: `customers_with_quick_support` captures the count of users contacting support within two weeks, and `support_summary` details activity by plan/region.
